# homuncu_loc image viewer

In [1]:
import napari
from skimage import io
import btrack
import os
from homuncu_loc import dataio
from glob import glob

/home/dayn/miniconda3/envs/homunc/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Define base directory and image ID

In [60]:
search_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/' # search for image or input basename and dir 
image_base_name = '20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504.tif' 
img_base_dir = dataio.find_image_directory(search_dir, image_base_name) # img_base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/images'
image_fn = os.path.join(img_base_dir, image_base_name)
image_id = dataio.ID_extractor(image_fn)
print(f"File '{image_base_name}' {'exists.' if os.path.exists(image_fn) else 'does not exist.'}")

File '20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504.tif' exists.


### Load image

In [61]:
%%time
image = io.imread(image_fn)
print(f'Image of shape {image.shape} loaded')

Image of shape (81, 2304, 2304, 4) loaded
CPU times: user 1.18 s, sys: 3.13 s, total: 4.31 s
Wall time: 1min 2s


### Search for associated single-cell segmentation and tracking data

In [63]:
# base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses'
sc_base_dir = img_base_dir.replace('images', 'sc_analyses') 
sc_fns = glob(os.path.join(sc_base_dir, f'*_{image_id}*.h5'))
print(sc_fns)

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_mphi.h5', '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_iat2.h5', '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_iat1.h5']


### Load single-cell data

In [64]:
sc_fn = sc_fns[0]
# load labels (tracks and segmentation)
with btrack.io.HDF5FileHandler(sc_fn, 
                                   'r', 
                                       obj_type='obj_type_1'#'obj_type_2'
                                   ) as reader:
        # tracks = [t for t in reader.tracks if np.mean(t.properties['area']) > 1000]
        tracks = [t for t in reader.tracks if len(t) >= 3]
        segmentation = reader.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)

[INFO][2023/11/03 02:14:48 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_mphi.h5...
[INFO][2023/11/03 02:14:49 pm] Loading tracks/obj_type_1
[INFO][2023/11/03 02:14:49 pm] Loading LBEP/obj_type_1
[INFO][2023/11/03 02:14:49 pm] Loading objects/obj_type_1 (28388, 5) (28388 filtered: None)
[INFO][2023/11/03 02:14:52 pm] Loading segmentation (81, 2304, 2304)
[INFO][2023/11/03 02:14:52 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_LDO_MTB/sc_analyses/20230802_20X_23-03-001A2_DAPI_ZO-1_LDO_Mtb_Multichannel Z-Stack_20230802_1504_mphi.h5


##### Optionally recolour segmentation according to Cell ID

In [65]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

### View images and single-cell data

In [66]:
viewer = napari.Viewer(title = f'Image ID: {image_id}')

viewer.add_image(image, channel_axis = -1)
# viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f53a389a5b0>

In [67]:
print()

In [27]:
image.shape

(127, 2304, 2304, 4)

In [28]:
mtb = image[...,-1]

In [29]:
ldo = image[...,-2]

In [34]:
import numpy as np

In [ ]:
np.

In [35]:
mtb.dtype

dtype('<u2')

In [36]:
test = (mtb-ldo).astype(np.uint16)

In [37]:
viewer.add_image(test)

<Image layer 'test [1]' at 0x7f53ab641eb0>